In [1]:
import requests
import json
from bs4 import BeautifulSoup as bs
import time
import random
import re
import pandas as pd
import os

In [27]:
from nltk import word_tokenize, pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords

In [2]:
from gensim.models.wrappers import FastText
model = FastText.load_fasttext_format('/Users/nigula/input/fr/fr.bin')


In [3]:
import numpy as np
from tqdm import tqdm

In [6]:
model['maman'].shape

(300,)

In [5]:
soup = bs(open("response.xls"), "html.parser")
first_string_passed = False
senseword_list = []
pos = ''
for link in soup.find_all("a", attrs={"class" : "translation"}):
    #print(link.prettify())
    try:
        #print("||",link['data-pos'])
        pos = link['data-pos'][1:-1]
    except:
        pass
    if first_string_passed == True:
        sence_word = link.text.strip()
        senseword_list.append({"sence_word":sence_word.lower(), "pos":pos})
        #print(sence_word)
    first_string_passed = True
if len (senseword_list) == 0:
    print("turn to alternative marks")
    for link in soup.find_all("div", attrs={"lang" : "fr"}):
        try:
            #print("||",link['data-pos'])
            pos = link['data-pos'][1:-1]
        except:
            pass
        sence_word = link.text.strip()
        senseword_list.append({"sence_word":sence_word.lower(), "pos":pos})
print(senseword_list)

[{'sence_word': 'devant', 'pos': ''}]


In [20]:
def send_request_func(from_lang,to_lang, word, header_main, login, en_from_lang, en_to_lang):
    url = "https://context.reverso.net/перевод/" + from_lang + "-" + to_lang + "/" + word 
    print(url)
    response = requests.get(url, headers=header_main, data = login)
    response.encoding = 'utf-8' 
    #
    save_dir = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/reverso" + "_" + en_from_lang + "_" + en_to_lang,word + ".xls")
    file = open(save_dir, "wb")
    file.write(response.content)
    file.close()
    soup = bs(response.text, 'html.parser') 
    return soup

In [30]:
def get_definitions_reverso(word, from_lang, to_lang, print_output = False, send_request = True):
    login = {'inUserName': 'n.babakov@lingualeo.com', 'inUserPass': '33vec33'}
    header_main = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    senseword_list = []
    land_dict = {"русский":"ru","французский":"fr", "английский":"en"}
    try:
        word_dir = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/reverso" + "_" + land_dict[from_lang] + "_" + land_dict[to_lang],word + ".xls")
        infile = open(word_dir,"r")
        contents = infile.read()
        soup = bs(contents,'html.parser')
    except:
        soup = send_request(from_lang,to_lang, word, header_main, login, land_dict[from_lang], land_dict[to_lang])
    #print(soup.prettify())
    first_string_passed = False
    pos = ''
    for link in soup.find_all("a", attrs={"class" : "translation"}):
        #print(link.prettify())
        try:
            #print("||",link['data-pos'])
            pos = link['data-pos'][1:-1]
        except:
            pass
        if first_string_passed == True:
            sence_word = link.text.strip()
            senseword_list.append({"sence_word":sence_word.lower(), "pos":pos})
            #print(sence_word)
        first_string_passed = True
    if len (senseword_list) == 0:
        print("turn to alternative marks")
        for link in soup.find_all("div", attrs={"lang" : "fr"}):
            try:
                #print("||",link['data-pos'])
                pos = link['data-pos'][1:-1]
            except:
                pass
            sence_word = link.text.strip()
            senseword_list.append({"sence_word":sence_word.lower(), "pos":pos})
            #print(link.prettify())
    time.sleep(2)
    return senseword_list
    """
    for transl_word in translations_variants:
        time.sleep(random.uniform(0.4,2.9))
        url_example = "https://context.reverso.net/перевод/" + to_lang + "-" + from_lang + "/" + transl_word
        response_ex = requests.get(url_example, headers=header_main)
        soup_sub = bs(response_ex.text, 'html.parser')
        print("========================================")
        print("\nEXAMPLES FOR SENCE ",transl_word,"\n" )
        for l in soup_sub.find_all('span', attrs = {'class':'text', 'lang':'en'}):
            example = l.text.strip()
            #print(example, word in example, type(l.text.strip()), word)
            regex = "\W" + word + "\w{0,2}\W"
            find = re.findall(regex, example)
            if len(find) > 0 :
                print(example)
    """

get_definitions_reverso("пред", "русский", "французский", print_output = False, send_request= False)

[{'sence_word': 'devant', 'pos': ''}]

In [23]:
get_definitions_reverso("devant", "французский", "русский", print_output = False)

https://context.reverso.net/перевод/французский-русский/devant


[{'sence_word': 'перед', 'pos': 'adv'},
 {'sence_word': 'впереди', 'pos': 'adv'},
 {'sence_word': 'вперед', 'pos': 'adv'},
 {'sence_word': 'спереди', 'pos': 'adv'},
 {'sence_word': 'снаружи', 'pos': 'adv'},
 {'sence_word': 'прямо', 'pos': 'adv'},
 {'sence_word': 'рядом с', 'pos': 'adv'},
 {'sence_word': 'рассмотрение', 'pos': 'n'},
 {'sence_word': 'мимо', 'pos': 'n'},
 {'sence_word': 'возле', 'pos': 'n'},
 {'sence_word': 'передо', 'pos': 'n'},
 {'sence_word': 'которые должны', 'pos': 'n'},
 {'sence_word': 'напротив', 'pos': 'n'},
 {'sence_word': 'суде', 'pos': 'n'},
 {'sence_word': 'пред', 'pos': 'n'}]

In [56]:
yandex_pos_set = {'adjective',
 'adverb',
 'conjunction',
 'foreign word',
 'no_pos_available',
 'noun',
 'numeral',
 'participle',
 'particle',
 'predicative',
 'preposition',
 'pronoun',
 'verb'}
yandex_pos_set

{'adjective',
 'adverb',
 'conjunction',
 'foreign word',
 'no_pos_available',
 'noun',
 'numeral',
 'participle',
 'particle',
 'predicative',
 'preposition',
 'pronoun',
 'verb'}

In [ ]:
contextrevers2yandex_dict = {'adj':"adjective", "v":"verb", "adv":"adverb", "v":"verb"}

In [ ]:
|| [adj]
chaud
|| [adj]
chaleureux
|| [adj]
tiède
|| [adj]
doux
|| [adj/nm]
bon
|| [adj]
cordial
|| [v]
réchauffer
|| [v]
chauffer
|| [v]
échauffer
|| [v]
se réchauffer
|| [v]
tiédir
|| [nf]
chaleur
|| [nm]
accueil
|| [adv]
chaleureusement
|| [adv]
Warm
accueillante
conviviale
['chaud',
 'chaleureux',
 'tiède',
 'doux',
 'bon',
 'cordial',
 'réchauffer',
 'chauffer',
 'échauffer',
 'se réchauffer',
 'tiédir',
 'chaleur',
 'accueil',
 'chaleureusement',
 'Warm',
 'accueillante',
 'conviviale']

In [15]:
get_definitions_reverso("warm", "английский", "французский", print_output = False)

[{'sence_word': 'chaud', 'pos': 'adj'},
 {'sence_word': 'chaleureux', 'pos': 'adj'},
 {'sence_word': 'tiède', 'pos': 'adj'},
 {'sence_word': 'doux', 'pos': 'adj'},
 {'sence_word': 'bon', 'pos': 'adj/nm'},
 {'sence_word': 'cordial', 'pos': 'adj'},
 {'sence_word': 'réchauffer', 'pos': 'v'},
 {'sence_word': 'chauffer', 'pos': 'v'},
 {'sence_word': 'échauffer', 'pos': 'v'},
 {'sence_word': 'se réchauffer', 'pos': 'v'},
 {'sence_word': 'tiédir', 'pos': 'v'},
 {'sence_word': 'chaleur', 'pos': 'nf'},
 {'sence_word': 'accueil', 'pos': 'nm'},
 {'sence_word': 'chaleureusement', 'pos': 'adv'},
 {'sence_word': 'warm', 'pos': 'adv'},
 {'sence_word': 'accueillante', 'pos': 'adv'},
 {'sence_word': 'conviviale', 'pos': 'adv'}]

In [16]:
df_en_ru_yandex = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_en_ru_new_vs_pos.csv")
df_en_ru_yandex.head()

,Unnamed: 0,word,local_word,pos,examples,local_examples
0,0,before,до,preposition,['before the war'],['до войны']
1,1,before,впереди,preposition,[],[]
2,2,before,пред,preposition,[],[]
3,3,before,накануне,preposition,[],[]
4,4,before,передо,preposition,[],[]


In [24]:
df_fr_en_yandex = pd.read_csv("/Users/nigula/LL/adjust_unigr_dict/lookup results/yandex_lookup_fr_en.csv")
df_fr_en_yandex.head()

,Unnamed: 0,word,local_word,pos,examples,local_examples
0,0,tomber,fall,verb,[],[]
1,1,tomber,fell,noun,[],[]
2,2,tomber,fall down,verb,[],[]
3,3,pénétrer,penetrate,verb,[],[]
4,4,complet,full,noun,[],[]


In [19]:
df_en_ru_yandex.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 5 columns):
Unnamed: 0        1218 non-null int64
word              1218 non-null object
local_word        1218 non-null object
examples          1218 non-null object
local_examples    1218 non-null object
dtypes: int64(1), object(4)
memory usage: 47.7+ KB


In [25]:
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
lemmatizer = FrenchLefffLemmatizer()

In [26]:
from  sklearn.metrics.pairwise import cosine_similarity

In [19]:
fr_list = ['mère', 'maman', 'mère de famille', 
           'materner', 'Vierge', 'Mother', 'parents', 'femme', 'maternel', 'fils', 'Dame', 'Marie']

In [43]:

def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return None
    
def get_nonstopwords_vectors_sum(lemm_list):
    orig_vector = np.zeros(shape = 100)
    for lemma in lemm_list:
        if lemma not in stopWords:
            #print (lemma)
            try:
                orig_vector += model[lemma]
            except:
                pass
    return orig_vector


def en_lemmatize(line):
    pos_tagged_ngramm = pos_tag(line.split())
    lemmatized_line_list = []
    for word_el in pos_tagged_ngramm:
        pos = get_wordnet_pos(word_el[1])
        if pos:
            lemma = lemmatizer.lemmatize(word_el[0], pos =pos)
        else:
            lemma = word_el[0]
        lemmatized_line_list.append(lemma)
    return ' '.join(lemmatized_line_list)

In [44]:
en_lemmatize("hello brothers")

'hello brother'

In [45]:
def fr_lemmatize(word):
    lemma = lemmatizer.lemmatize(word)
    return lemma
def preprocess_words(words_list, lemmatizer_func):
    preprocessed_words = []
    for word in words_list:
        lemma = lemmatizer_func(word['sence_word'].lower())
        word['sence_word'] = lemma
        #print("lemma",lemma)
        #preprocessed_words.append(lemma)
    #return preprocessed_words
        

In [29]:
l1 = "maman"
l2 = 'maman de famille'

def get_vectors_sum(lemm_line):
    orig_vector = np.zeros(shape = 300)
    for lemma in lemm_line.split():
        #print(lemma)
        try:
            orig_vector += model[lemma]
        except:
            pass
    return orig_vector

def find_lines_similarity(line_1, line_2):
    l1_vect = get_vectors_sum(line_1)
    l2_vect = get_vectors_sum(line_2)
    cos_sim = cosine_similarity(l1_vect.reshape(1, -1), l2_vect.reshape(1, -1))[0][0]
    #print(cos_sim)
    return cos_sim

find_lines_similarity(l1, l2)      

0.7949875776604262

In [38]:
def compare_pos(pos1, pos2):
    if pos1 == pos2:
        #print("abs eq", pos1, pos2)
        return True
    elif pos1 in pos2 or pos2 in pos1:
        #print("inclusive eq", pos1, pos2)
        return True
    #print("no eq", pos1, pos2)
    return False

In [39]:
def equate_empty_pos(pos1, pos2):
    if pos1 == '': 
        pos1 = pos2
        #print("pos1 eqauted")
    elif pos2 == '': 
        pos2 = pos1
        #print("pos2 eqauted")
    return pos1, pos2

In [40]:
def get_two_target_lang_lists_intersection(list_1, list_2):
    overall_intersection = []
    cos_sim_dict ={}
    #overall_intersection
    for word_1 in list_1:
        for word_2 in list_2:
            #print(word_1, word_2)
            word_1['pos'], word_2['pos'] = equate_empty_pos(word_1['pos'], word_2['pos'])
            pos_eq = compare_pos(word_1['pos'],word_2['pos'])  
            if pos_eq == True and word_1['sence_word'] == word_2['sence_word']:
                #overall_intersection.append(word_1)
                cos_sim_dict[word_1['sence_word']] = []
                
                
    #handle indirect similarity
    handled_words = []
    for word_1 in list_1:
        for word_2 in list_2:
            word_1['pos'], word_2['pos'] = equate_empty_pos(word_1['pos'], word_2['pos'])
            pos_eq = compare_pos(word_1['pos'],word_2['pos'])
            if pos_eq == True and word_1['sence_word'] != word_2['sence_word'] and word_1['sence_word'] not in handled_words and word_2['sence_word'] not in handled_words:
                sim = find_lines_similarity(word_1['sence_word'], word_2['sence_word'])
                if sim > 0.9:
                    print('COS_SIM_WORKED', word_1['sence_word'], word_2['sence_word'])
                    if word_1['sence_word'] in cos_sim_dict:
                        handled_words.append(word_2)
                        cos_sim_dict[word_1['sence_word']].append(word_2['sence_word'])
                       # overall_intersection[word_1_index] += '| ' + word_2
                    
                    elif word_2['sence_word'] in cos_sim_dict:
                         handled_words.append(word_1['sence_word'])
                         cos_sim_dict[word_2['sence_word']].append(word_1['sence_word'])
                    else:
                        cos_sim_dict[word_1['sence_word']] = [word_2['sence_word']]
                        #new_pair = word_1 + " | " + word_2
                        #overall_intersection.append(new_pair)
    #print(cos_sim_dict)
    
    #GETET INTERSECTION LIST
    for first_word in cos_sim_dict.keys():
        intersect_element = first_word
        for similar_word in cos_sim_dict[first_word]:
            intersect_element += " | " + similar_word
        overall_intersection.append(intersect_element)
    return overall_intersection

fr_proc_1 = [{'sence_word': 'réchauffer', 'pos': 'v'}, {'sence_word': 'chaud', 'pos': 'v'}]

fr_proc_2 = [{'sence_word': 'chaud', 'pos': 'adj'},
 {'sence_word': 'chaleureux', 'pos': 'adj'},
 {'sence_word': 'tiède', 'pos': 'adj'},
 {'sence_word': 'doux', 'pos': 'adj'},
 {'sence_word': 'bon', 'pos': 'adj/nm'},
 {'sence_word': 'cordial', 'pos': 'adj'},
 {'sence_word': 'réchauffer', 'pos': 'v'},
 {'sence_word': 'chauffer', 'pos': 'v'},
 {'sence_word': 'échauffer', 'pos': 'v'},
 {'sence_word': 'se réchauffer', 'pos': 'v'},
 {'sence_word': 'tiédir', 'pos': 'v'},
 {'sence_word': 'chaleur', 'pos': 'nf'},
 {'sence_word': 'accueil', 'pos': 'nm'},
 {'sence_word': 'chaleureusement', 'pos': 'adv'},
 {'sence_word': 'warm', 'pos': 'adv'},
 {'sence_word': 'accueillante', 'pos': 'adv'},
 {'sence_word': 'conviviale', 'pos': 'adv'}]
get_two_target_lang_lists_intersection(fr_proc_1, fr_proc_2)      
            

COS_SIM_WORKED réchauffer chauffer
COS_SIM_WORKED réchauffer échauffer


['réchauffer | chauffer | échauffer']

In [83]:
a = [{'sence_word': 'réchauffer', 'pos': 'v'}, {'sence_word': 'chaud', 'pos': 'v'}]
b= [{'sence_word': 'chaud', 'pos': 'adj'}, {'sence_word': 'chaleureux', 'pos': 'adj'}, 
    {'sence_word': 'tiède', 'pos': 'adj'}, {'sence_word': 'doux', 'pos': 'adj'}, {'sence_word': 'bon', 'pos': 'adj/nm'}, {'sence_word': 'cordial', 'pos': 'adj'}, {'sence_word': 'réchauffer', 'pos': 'v'}, {'sence_word': 'chauffer', 'pos': 'v'}, {'sence_word': 'échauffer', 'pos': 'v'}, {'sence_word': 'se réchauffer', 'pos': 'v'}, {'sence_word': 'tiédir', 'pos': 'v'}, {'sence_word': 'chaleur', 'pos': 'nf'}, {'sence_word': 'accueil', 'pos': 'nm'}, {'sence_word': 'chaleureusement', 'pos': 'adv'}, {'sence_word': 'warm', 'pos': 'adv'}, 
    {'sence_word': 'accueillante', 'pos': 'adv'}, {'sence_word': 'conviviale', 'pos': 'adv'}]
get_two_target_lang_lists_intersection(a, b)      


['réchauffer | chauffer | échauffer']

In [ ]:
fr_proc_1 = [{'sence_word': 'réchauffer', 'pos': 'v'}, {'sence_word': 'chaud', 'pos': 'v'}]
fr_proc_2 = [{'sence_word': 'réchauffer', 'pos': 'v'}, {'sence_word': 'chaud', 'pos': 'v'}]

In [17]:
def get_definitions_yand(word, lang):
    if lang == "ru":
        word_path = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_lookup_en_ru/", word + ".json")
    elif lang == "en":
        word_path = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_en_fr_whole_list/", word + ".json")
    local_word = []
    with open(word_path, "r") as f: 
        data = json.load(f)
        for definition in data['def']:
            word_current =  definition['text']
            for translation in definition['tr']: 
                local_word.append(translation['text'])
    return local_word

In [33]:
def get_cross_translate_yand(word_lang_1, lang_1, word_lang_2, lang_2, target_lang, lemmatizer_func):
    lang_1_to_target_words_list = get_definitions(word_lang_1, lang_1)
    lang_1_processed = preprocess_words(lang_1_to_target_words_list, lemmatizer_func)
    print(lang_1_processed)
    lang_2_to_target_words_list = get_definitions(word_lang_2, lang_2)
    lang_2_processed = preprocess_words(lang_2_to_target_words_list, lemmatizer_func)
    print(lang_2_processed)
    intersected_elements = get_two_target_lang_lists_intersection(lang_1_processed, lang_2_processed)
    if len(intersected_elements) == 0:intersected_elements.append("no_equality")
    return intersected_elements

get_cross_translate_yand("warm", "en", "согревать","ru", "fr", fr_lemmatize)


NameError: name 'get_definitions' is not defined

In [69]:
get_definitions_reverso("warm", "английский", "французский", print_output = False)

[{'sence_word': 'chaud', 'pos': 'adj'},
 {'sence_word': 'chaleureux', 'pos': 'adj'},
 {'sence_word': 'tiède', 'pos': 'adj'},
 {'sence_word': 'doux', 'pos': 'adj'},
 {'sence_word': 'bon', 'pos': 'adj/nm'},
 {'sence_word': 'cordial', 'pos': 'adj'},
 {'sence_word': 'réchauffer', 'pos': 'v'},
 {'sence_word': 'chauffer', 'pos': 'v'},
 {'sence_word': 'échauffer', 'pos': 'v'},
 {'sence_word': 'se réchauffer', 'pos': 'v'},
 {'sence_word': 'tiédir', 'pos': 'v'},
 {'sence_word': 'chaleur', 'pos': 'nf'},
 {'sence_word': 'accueil', 'pos': 'nm'},
 {'sence_word': 'chaleureusement', 'pos': 'adv'},
 {'sence_word': 'warm', 'pos': 'adv'},
 {'sence_word': 'accueillante', 'pos': 'adv'},
 {'sence_word': 'conviviale', 'pos': 'adv'}]

In [70]:
get_definitions_reverso("согревать", "русский", "французский", print_output = False)

turn to alternative marks


[{'sence_word': 'réchauffer', 'pos': 'v'}, {'sence_word': 'chaud', 'pos': 'v'}]

In [88]:
get_two_target_lang_lists_intersection([{'sence_word': 'chaud', 'pos': 'adj'},
 {'sence_word': 'chaleureux', 'pos': 'adj'},
 {'sence_word': 'tiède', 'pos': 'adj'},
 {'sence_word': 'doux', 'pos': 'adj'},
 {'sence_word': 'bon', 'pos': 'adj/nm'},
 {'sence_word': 'cordial', 'pos': 'adj'},
 {'sence_word': 'réchauffer', 'pos': 'v'},
 {'sence_word': 'chauffer', 'pos': 'v'},
 {'sence_word': 'échauffer', 'pos': 'v'},
 {'sence_word': 'se réchauffer', 'pos': 'v'},
 {'sence_word': 'tiédir', 'pos': 'v'},
 {'sence_word': 'chaleur', 'pos': 'nf'},
 {'sence_word': 'accueil', 'pos': 'nm'},
 {'sence_word': 'chaleureusement', 'pos': 'adv'},
 {'sence_word': 'warm', 'pos': 'adv'},
 {'sence_word': 'accueillante', 'pos': 'adv'},
 {'sence_word': 'conviviale', 'pos': 'adv'}], 
                                      
[{'sence_word': 'réchauffer', 'pos': 'v'}, {'sence_word': 'chaud', 'pos': 'v'}]                            )

['réchauffer', 'chauffer | réchauffer', 'échauffer | réchauffer']

In [29]:
get_two_target_lang_lists_intersection([{'sence_word': 'chaud', 'pos': 'adj'}, 
                                        {'sence_word': 'chaleureux', 'pos': 'adj'}, {'sence_word': 'tiède', 'pos': 'adj'}, 
                                        {'sence_word': 'doux', 'pos': 'adj'}, {'sence_word': 'bon', 'pos': 'adj/nm'}, 
                                        {'sence_word': 'cordial', 'pos': 'adj'}, {'sence_word': 'réchauffer', 'pos': 'v'}, 
                                        {'sence_word': 'chauffer', 'pos': 'v'}, {'sence_word': 'échauffer', 'pos': 'v'}, 
                                        {'sence_word': 'se réchauffer', 'pos': 'v'}, {'sence_word': 'tiédir', 'pos': 'v'}, 
                                        {'sence_word': 'chaleur', 'pos': 'nf'}, {'sence_word': 'accueil', 'pos': 'nm'}, 
                                        {'sence_word': 'chaleureusement', 'pos': 'adv'}, {'sence_word': 'warm', 'pos': 'adv'}, 
                                        {'sence_word': 'accueillante', 'pos': 'adv'}, {'sence_word': 'conviviale', 'pos': 'adv'}], 
                                       
                                       [{'sence_word': 'réchauffer', 'pos': 'v'}, {'sence_word': 'chaud', 'pos': 'v'}])



['réchauffer | chauffer | échauffer']

In [46]:
def get_cross_translate_reverso(word_lang_1, lang_1, word_lang_2, lang_2, target_lang, lemmatizer_func):
    lang_1_to_target_words_dict = get_definitions_reverso(word_lang_1, lang_1, target_lang, print_output=True)
    preprocess_words(lang_1_to_target_words_dict, lemmatizer_func)
    
    
    lang_2_to_target_words_dict = get_definitions_reverso(word_lang_2, lang_2, target_lang, print_output=True)
    preprocess_words(lang_2_to_target_words_dict, lemmatizer_func)
    
    print("lang_1_to_target_words_dict", lang_1_to_target_words_dict,"\n")
    print("lang_2_to_target_words_dict", lang_2_to_target_words_dict)
    
    intersected_elements = get_two_target_lang_lists_intersection(lang_1_to_target_words_dict, lang_2_to_target_words_dict)
    
    if len(intersected_elements) == 0:intersected_elements.append("no_equality")
    return intersected_elements

get_cross_translate_reverso("warm", "английский", "согревать","русский", "французский", fr_lemmatize)


turn to alternative marks
lang_1_to_target_words_dict [{'sence_word': 'chaud', 'pos': 'adj'}, {'sence_word': 'chaleureux', 'pos': 'adj'}, {'sence_word': 'tiède', 'pos': 'adj'}, {'sence_word': 'doux', 'pos': 'adj'}, {'sence_word': 'bon', 'pos': 'adj/nm'}, {'sence_word': 'cordial', 'pos': 'adj'}, {'sence_word': 'réchauffer', 'pos': 'v'}, {'sence_word': 'chauffer', 'pos': 'v'}, {'sence_word': 'échauffer', 'pos': 'v'}, {'sence_word': 'se réchauffer', 'pos': 'v'}, {'sence_word': 'tiédir', 'pos': 'v'}, {'sence_word': 'chaleur', 'pos': 'nf'}, {'sence_word': 'accueil', 'pos': 'nm'}, {'sence_word': 'chaleureusement', 'pos': 'adv'}, {'sence_word': 'warm', 'pos': 'adv'}, {'sence_word': 'accueillante', 'pos': 'adv'}, {'sence_word': 'conviviale', 'pos': 'adv'}] 

lang_2_to_target_words_dict [{'sence_word': 'réchauffer', 'pos': 'v'}, {'sence_word': 'chaud', 'pos': 'v'}]
COS_SIM_WORKED chauffer réchauffer
COS_SIM_WORKED échauffer réchauffer


['réchauffer | chauffer | échauffer']

In [31]:
df_en_ru_yandex.head()

,Unnamed: 0,word,local_word,pos,examples,local_examples
0,0,before,до,preposition,['before the war'],['до войны']
1,1,before,впереди,preposition,[],[]
2,2,before,пред,preposition,[],[]
3,3,before,накануне,preposition,[],[]
4,4,before,передо,preposition,[],[]


In [33]:
def get_multilang_from_df(df):
    eng_word_list = []
    lang_1_word_list = []
    lang_2_word_list = []
    eng_word = 'first_word'
    for df_ind in tqdm(range(len(df))):
        df_element = list(df.iloc[df_ind])
        if eng_word != df_element[1]: print("\n************************************\n")
        eng_word = df_element[1]
        loc_1_word = df_element[2]
        print(eng_word, loc_1_word)
        sense_intersect = get_cross_translate_reverso(eng_word, "английский", loc_1_word,"русский", "французский", fr_lemmatize)
        #sense_intersect = get_cross_translate_yand(eng_word, "en", loc_1_word,"ru", "fr", fr_lemmatize)
        print("\n SENSE_INTERSECTION ", sense_intersect)
        for sense_intersect_element in sense_intersect:
            eng_word_list.append(eng_word)
            lang_1_word_list.append(loc_1_word)
            lang_2_word_list.append(sense_intersect_element)
            
        
        if df_ind % 10 == 0:
            save_path = os.path.join("/Users/nigula/LL/adjust_unigr_dict/reverse_connected_dicts/cross_barbos",
                         str(df_ind) + ".csv")
            data = pd.DataFrame(list(zip(eng_word_list, lang_1_word_list,lang_2_word_list)),
                        columns =['eng_word', 'rus','fr'])
            data.to_csv(save_path)
            
    data = pd.DataFrame(list(zip(eng_word_list, lang_1_word_list,lang_2_word_list)),
                        columns =['eng_word', 'rus','fr'])
    
    return data

df_din = get_multilang_from_df(df_en_ru_yandex)
df_din.head()


  0%|          | 0/1218 [00:00<?, ?it/s]


************************************

before до
https://context.reverso.net/перевод/английский-французский/before
https://context.reverso.net/перевод/русский-французский/до



  0%|          | 1/1218 [00:05<1:44:52,  5.17s/it]

lang_1_to_target_words_dict [{'sence_word': 'avant', 'pos': 'adv'}, {'sence_word': 'avant que', 'pos': 'adv'}, {'sence_word': 'déjà', 'pos': 'adv'}, {'sence_word': 'tôt', 'pos': 'adv'}, {'sence_word': 'précédemment', 'pos': 'adv'}, {'sence_word': 'heure', 'pos': 'adv'}, {'sence_word': 'préalablement', 'pos': 'adv'}, {'sence_word': 'antérieurement', 'pos': 'adv'}, {'sence_word': 'préalablement à', 'pos': 'adv'}, {'sence_word': 'devant', 'pos': 'adv'}, {'sence_word': 'auparavant', 'pos': 'adv'}, {'sence_word': 'dont', 'pos': 'adv'}, {'sence_word': 'précédant', 'pos': 'adv'}, {'sence_word': 'précédent', 'pos': 'adv'}, {'sence_word': 'précédé', 'pos': 'adv'}, {'sence_word': 'présenté', 'pos': 'adv'}, {'sence_word': 'passé', 'pos': 'adv'}, {'sence_word': 'juste avant', 'pos': 'adv'}, {'sence_word': 'soumise', 'pos': 'adv'}] 

lang_2_to_target_words_dict [{'sence_word': 'avant', 'pos': ''}, {'sence_word': "jusqu'à", 'pos': ''}, {'sence_word': "jusqu'au", 'pos': ''}, {'sence_word': 'tant', 'p


  0%|          | 2/1218 [00:10<1:43:21,  5.10s/it]

...


100%|█████████▉| 1217/1218 [1:42:24<00:04,  4.94s/it]

lang_1_to_target_words_dict [{'sence_word': 'lire', 'pos': 'v'}, {'sence_word': 'consulter', 'pos': 'v'}, {'sence_word': 'voir', 'pos': 'v'}, {'sence_word': 'relire', 'pos': 'v'}, {'sence_word': 'interpréter', 'pos': 'v'}, {'sence_word': 'déchiffrer', 'pos': 'v'}, {'sence_word': 'indiquer', 'pos': 'v'}, {'sence_word': 'bouquiner', 'pos': 'v'}, {'sence_word': 'pris connaissance', 'pos': 'pp'}, {'sence_word': 'libellé', 'pos': 'pp'}, {'sence_word': 'lecture', 'pos': 'pp'}, {'sence_word': 'recevez', 'pos': 'pp'}, {'sence_word': 'reçois', 'pos': 'pp'}, {'sence_word': 'lisible', 'pos': 'pp'}, {'sence_word': 'relever', 'pos': 'v'}, {'sence_word': 'read', 'pos': 'v'}, {'sence_word': 'en savoir', 'pos': 'v'}, {'sence_word': 'retrouvez', 'pos': 'v'}] 

lang_2_to_target_words_dict []

 SENSE_INTERSECTION  ['no_equality']
read чтение
https://context.reverso.net/перевод/английский-французский/read
https://context.reverso.net/перевод/русский-французский/чтение



100%|██████████| 1218/1218 [1:42:29<00:00,  4.92s/it]

lang_1_to_target_words_dict [{'sence_word': 'lire', 'pos': 'v'}, {'sence_word': 'consulter', 'pos': 'v'}, {'sence_word': 'voir', 'pos': 'v'}, {'sence_word': 'relire', 'pos': 'v'}, {'sence_word': 'interpréter', 'pos': 'v'}, {'sence_word': 'déchiffrer', 'pos': 'v'}, {'sence_word': 'indiquer', 'pos': 'v'}, {'sence_word': 'bouquiner', 'pos': 'v'}, {'sence_word': 'pris connaissance', 'pos': 'pp'}, {'sence_word': 'libellé', 'pos': 'pp'}, {'sence_word': 'lecture', 'pos': 'pp'}, {'sence_word': 'recevez', 'pos': 'pp'}, {'sence_word': 'reçois', 'pos': 'pp'}, {'sence_word': 'lisible', 'pos': 'pp'}, {'sence_word': 'relever', 'pos': 'v'}, {'sence_word': 'read', 'pos': 'v'}, {'sence_word': 'en savoir', 'pos': 'v'}, {'sence_word': 'retrouvez', 'pos': 'v'}] 

lang_2_to_target_words_dict [{'sence_word': 'lecture', 'pos': 'nf'}, {'sence_word': 'lire', 'pos': 'nf'}]

 SENSE_INTERSECTION  ['no_equality']


,eng_word,rus,fr
0,before,до,avant
1,before,до,auparavant
2,before,до,précédant
3,before,впереди,devant
4,before,пред,devant


In [121]:
a = [{'sence_word': 'tambour', 'pos': 'nm'}, {'sence_word': 'batterie', 'pos': 'nf'}, {'sence_word': 'cylindre', 'pos': 'nm'}, {'sence_word': 'drum', 'pos': 'n'}, {'sence_word': 'baril', 'pos': 'nm'}, {'sence_word': 'bidon', 'pos': 'nm'}, {'sence_word': 'tonneau', 'pos': 'nm'}, {'sence_word': 'enrouleur', 'pos': 'nm'}, {'sence_word': 'barillet', 'pos': 'nm'}, {'sence_word': 'percussion', 'pos': 'nf'}, {'sence_word': 'rouleau', 'pos': 'nf'}]
b= [{'sence_word': 'baril', 'pos': 'n'}]

get_two_target_lang_lists_intersection(a, b)


['baril']

In [109]:
a = [{'sence_word': 'avant', 'pos': 'adv'}, {'sence_word': 'avant que', 'pos': 'adv'}, {'sence_word': 'déjà', 'pos': 'adv'}, {'sence_word': 'tôt', 'pos': 'adv'}, {'sence_word': 'précédemment', 'pos': 'adv'}, {'sence_word': 'heure', 'pos': 'adv'}, {'sence_word': 'préalablement', 'pos': 'adv'}, {'sence_word': 'antérieurement', 'pos': 'adv'}, {'sence_word': 'préalablement à', 'pos': 'adv'}, {'sence_word': 'devant', 'pos': 'adv'}, {'sence_word': 'auparavant', 'pos': 'adv'}, {'sence_word': 'dont', 'pos': 'adv'}, {'sence_word': 'précédant', 'pos': 'adv'}, {'sence_word': 'précédent', 'pos': 'adv'}, {'sence_word': 'précédé', 'pos': 'adv'}, {'sence_word': 'présenté', 'pos': 'adv'}, {'sence_word': 'passé', 'pos': 'adv'}, {'sence_word': 'juste avant', 'pos': 'adv'}, {'sence_word': 'soumise', 'pos': 'adv'}] 
b = [{'sence_word': 'avant', 'pos': ''}, {'sence_word': "jusqu'à", 'pos': ''}, {'sence_word': "jusqu'au", 'pos': ''}, {'sence_word': 'tant', 'pos': ''}, {'sence_word': "jusqu'en", 'pos': ''}, {'sence_word': 'en attendant', 'pos': ''}, {'sence_word': 'atteint', 'pos': ''}, {'sence_word': 'auparavant', 'pos': ''}, {'sence_word': 'ver', 'pos': ''}, {'sence_word': 'maximum', 'pos': ''}, {'sence_word': 'précédant', 'pos': ''}, {'sence_word': 'pendant', 'pos': ''}, {'sence_word': 'près', 'pos': ''}, {'sence_word': 'environ', 'pos': ''}, {'sence_word': 'do', 'pos': ''}]

get_two_target_lang_lists_intersection(a, b)


pos2 eqauted
pos2 eqauted
pos2 eqauted
pos2 eqauted
pos2 eqauted
pos2 eqauted
pos2 eqauted
pos2 eqauted
pos2 eqauted
pos2 eqauted
pos2 eqauted
pos2 eqauted
pos2 eqauted
pos2 eqauted
pos2 eqauted


['avant', 'auparavant', 'précédant']

In [123]:
df_din.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1704 entries, 0 to 1703
Data columns (total 3 columns):
eng_word    1704 non-null object
rus         1704 non-null object
fr          1704 non-null object
dtypes: object(3)
memory usage: 40.0+ KB


In [1]:
df_din.head(100)

NameError: name 'df_din' is not defined

In [ ]:
#df_din.to_csv("cross_approach.csv")

In [125]:
df_din.to_csv("cross_approach_yandex_vs_contreverso_v3_vs_pos.csv")

# Y_LOOkup_azure_cross_aproach

In [ ]:
from io import StringIO

In [ ]:
df_en_ru_yandex.head()

In [ ]:
def get_stringfied_json(word):
    dt = {'Text': word}
    stringIO = StringIO()
    json.dump(dt, stringIO)
    return stringIO.getvalue()

In [ ]:
headers = {
    'Ocp-Apim-Subscription-Key': '7f44a7c95fd34041a2eef780f3f79817',
    'Content-Type': 'application/json',
}

params = (
    ('api-version', '3.0'),
    ('from', 'ru'),
    ('to', 'fr'),
)

In [ ]:
https://docs.microsoft.com/en-us/azure/cognitive-services/translator/language-support
    
    
    The dictionary supports the following languages to or from English using the Lookup and Examples methods.

In [ ]:
for local_word in list(df_en_ru_yandex['local_word']):
    print (local_word)
    data = get_stringfied_json(local_word)
    response = requests.post('https://api.cognitive.microsofttranslator.com/dictionary/lookup', headers=headers, params=params, data=data)
    save_loc = "./lookup/azure_ru_fr/" + local_word + ".json"
    result = json.loads(response.text)
    with open(save_loc, "w") as f:
        json.dump(result,f,indent = 4, ensure_ascii=False)
    break

# lang pair not available

# Y_LOOKUP+Y_LOOKUP approach

In [ ]:
def get_definitions(word, from_lang, to_lang, print_output = False):


In [ ]:
for local_word in list(df_en_ru_yandex['local_word']):
    print (local_word)
    url = "https://dictionary.yandex.net/api/v1/dicservice.json/lookup"
    headers = {"key":"dict.1.1.20190828T215834Z.0d8a3da5e08df3a0.85290ff28457230b81cb3b73720d2a0312ca14dc",
                  "lang": "ru-fr","text":local_word}
    result = requests.get(url, headers).json()
    save_loc = "./lookup/yandex_ru_fr/" + local_word + ".json"
    #result = json.loads(response.text)
    with open(save_loc, "w") as f:
        json.dump(result,f,indent = 4, ensure_ascii=False)
    

In [ ]:
def parse_json_vs_lists(path, word, local_word,examples,examples_local):
    try:
        with open(path, "r") as f: 
            data = json.load(f)
            for definition in data['def']:
                word_current =  definition['text']
                for translation in definition['tr']: 
                    word.append(word_current)
                    local_word.append(translation['text'])
                    ex_en = []
                    ex_rus = []
                    if 'ex' in translation:
                        for exmpl in translation['ex']:
                            #print(exmpl)
                            ex_en.append(exmpl['text'])
                            ex_rus.append(exmpl['tr'][0]['text'])
                    examples.append(ex_en)
                    examples_local.append(ex_rus)
    except:
        pass

In [ ]:
def get_definitions(word, lang):
    if lang == "ru":
        word_path = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_ru_fr/", word + ".json")
    elif lang == "fr":
        word_path = os.path.join("/Users/nigula/LL/adjust_unigr_dict/lookup/yandex_en_fr_whole_list/", word + ".json")
    local_word = []
    with open(word_path, "r") as f: 
        data = json.load(f)
        for definition in data['def']:
            word_current =  definition['text']
            for translation in definition['tr']: 
                local_word.append(translation['text'])
    return local_word

In [ ]:
get_definitions("право", "ru")

In [ ]:
def get_definitions(word, from_lang, to_lang,print_output = False):
    url = "https://dictionary.yandex.net/api/v1/dicservice.json/lookup"
    headers = {"key":"dict.1.1.20190828T215834Z.0d8a3da5e08df3a0.85290ff28457230b81cb3b73720d2a0312ca14dc",
                  "lang": from_lang + '-' to_lang,"text":word}
    ddd = requests.get(url, headers).json()
    ddd